In [74]:
import numpy as np
import matplotlib.pyplot as plt

# !pip install scikit-learn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

from collections import deque

from rashomon import loss
from rashomon import counter
from rashomon.aggregate import RAggregate
from rashomon.sets import RashomonSet, RashomonProblemCache, RashomonSubproblemCache
from rashomon import tva
from rashomon.extract_pools import extract_pools


%load_ext autoreload
%autoreload 2
# %matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Function to pool based on $\Sigma$ matrix

In [75]:
sigma = np.array([[1, 1, 0],
                  [0, 1, 1],
                  [1, 1, 1]], dtype='float64')

M, n = sigma.shape
R = np.array([5, 5, 5])

num_policies = np.prod(R-1)
policies = tva.enumerate_policies(M, R)
pi_pools, pi_policies = extract_pools(policies, sigma)

for pool_id, pool in pi_pools.items():
    print(pool_id, ":", [policies[i] for i in pool])

0 : [(1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 1, 4), (2, 1, 1), (2, 1, 2), (2, 1, 3), (2, 1, 4), (3, 1, 1), (3, 1, 2), (3, 1, 3), (3, 1, 4)]
1 : [(1, 2, 1), (1, 2, 2), (1, 2, 3), (1, 2, 4), (1, 3, 1), (1, 3, 2), (1, 3, 3), (1, 3, 4), (1, 4, 1), (1, 4, 2), (1, 4, 3), (1, 4, 4), (2, 2, 1), (2, 2, 2), (2, 2, 3), (2, 2, 4), (2, 3, 1), (2, 3, 2), (2, 3, 3), (2, 3, 4), (2, 4, 1), (2, 4, 2), (2, 4, 3), (2, 4, 4), (3, 2, 1), (3, 2, 2), (3, 2, 3), (3, 2, 4), (3, 3, 1), (3, 3, 2), (3, 3, 3), (3, 3, 4), (3, 4, 1), (3, 4, 2), (3, 4, 3), (3, 4, 4)]
2 : [(4, 1, 1), (4, 1, 2), (4, 1, 3), (4, 1, 4)]
3 : [(4, 2, 1), (4, 2, 2), (4, 2, 3), (4, 2, 4), (4, 3, 1), (4, 3, 2), (4, 3, 3), (4, 3, 4), (4, 4, 1), (4, 4, 2), (4, 4, 3), (4, 4, 4)]


### Generate data

In [3]:
np.random.seed(3)

num_pools = len(pi_pools)
# mu = np.random.uniform(0, 4, size=num_pools)
mu = np.array([3, 6, 1, 4])
var = [1] * num_pools

n_per_pol = 1000

num_data = num_policies * n_per_pol
X = np.ndarray(shape=(num_data, M))
D = np.ndarray(shape=(num_data, 1), dtype='int_')
y = np.ndarray(shape=(num_data, 1))

for idx, policy in enumerate(policies):
    pool_i = pi_policies[idx]
    mu_i = mu[pool_i]
    var_i = var[pool_i]
    y_i = np.random.normal(mu_i, var_i, size=(n_per_pol, 1))

    start_idx = idx * n_per_pol
    end_idx = (idx + 1) * n_per_pol

    X[start_idx:end_idx, ] = policy
    D[start_idx:end_idx, ] = idx
    y[start_idx:end_idx, ] = y_i
    

In [4]:
# This function needs to called only once
policy_means = loss.compute_policy_means(D, y, num_policies)

# This function needs to be called every time the pools change
mu_pools = loss.compute_pool_means(policy_means, pi_pools)

# This function needs to be called every time the pools change
Q = loss.compute_Q(D, y, sigma, policies, policy_means, 0.1)

print(Q)

1.3959767322482817


In [5]:
i = 0
j = 0

B = loss.compute_B(D, y, sigma, i, j, policies, policy_means, 0.1)
print(B)

1.3958936127485342


### RAggregate

In [30]:
P_set = RAggregate(M, R, 7, D, y, 2, reg=0.1)
print(P_set.size)
P_set.seen(sigma)

7


True

In [72]:
pol_means = loss.compute_policy_means(D, y, num_policies)
true_best = pi_pools[np.argmax(mu)]

for s_i in P_set:
    # print(s_i)
    pi_pools_i, pi_policies_i = extract_pools(policies, s_i)
    pool_means_i = loss.compute_pool_means(pol_means, pi_pools_i)
    
    Q = loss.compute_Q(D, y, s_i, policies, pol_means, reg=0.1)
    y_pred = loss.make_predictions(D, pi_policies_i, pool_means_i)
    sqrd_err = mean_squared_error(y, y_pred)
    pol_max = loss.find_best_policies(D, y_pred)
    iou = loss.intersect_over_union(set(true_best), set(pol_max))
    
    # print(np.max(y_pred), pool_means)
    print(f"Num pools: {len(pi_pools_i)}, Squared loss: {sqrd_err:0.5f}, Q: {Q:0.5f}")
    print(f"Best policy IOU: {iou:.3f}")
    print(f"Policy effects: {pool_means_i}")
    print("---")

Num pools: 2, Squared loss: 1.74172, Q: 1.94172
Best policy IOU: 0.750
Policy effects: [2.48562147 5.50364155]
---
Num pools: 4, Squared loss: 1.49805, Q: 1.89805
Best policy IOU: 0.667
Policy effects: [2.98462409 5.99545919 1.98661885 5.01182391]
---
Num pools: 4, Squared loss: 0.99598, Q: 1.39598
Best policy IOU: 1.000
Policy effects: [2.98650681 6.00144728 0.98296544 4.01022435]
---
Num pools: 6, Squared loss: 0.99594, Q: 1.59594
Best policy IOU: 0.333
Policy effects: [2.98462409 5.99545919 2.99027226 6.01342346 0.98296544 4.01022435]
---
Num pools: 6, Squared loss: 0.99591, Q: 1.59591
Best policy IOU: 0.667
Policy effects: [2.97088666 5.98855774 2.99431689 6.00789205 0.98296544 4.01022435]
---
Num pools: 6, Squared loss: 0.99590, Q: 1.59590
Best policy IOU: 0.667
Policy effects: [2.98650681 5.99153624 6.0064028  0.98296544 3.98828601 4.02119353]
---
Num pools: 6, Squared loss: 0.99590, Q: 1.59590
Best policy IOU: 0.333
Policy effects: [2.98650681 5.99430785 6.01572614 0.98296544 4.

### LASSO - Beta -> alpha

In [76]:
G = tva.alpha_matrix(M, R, policies)
# print(G)
D_matrix = tva.get_dummy_matrix(D, G, num_policies)

In [45]:
mod1 = linear_model.Lasso(alpha=1e-1)
mod1.fit(D_matrix, y)
y_tva = mod1.predict(D_matrix)
sqrd_err = mean_squared_error(y_tva, y)
print(sqrd_err)

1.102648733667545


In [65]:
# print(pi_pools[1])
# np.unique(D[np.where(y_tva == np.max(y_tva)), ])
tva_best = find_best_policies(D, y_tva)
iou_tva = intersect_over_union(set(true_best), set(tva_best))
print(iou_tva)

1.0


In [42]:
policies[11]

(1, 3, 4)

### Varying R

In [192]:
# Idea: np.inf for arbitrary dosage?
sigma = np.array([[1, 0, 1],
                  [0, 1, np.inf]], dtype='float64')
# sigma = np.array([[0, 0, 0],
#                   [0, 0, np.inf]], dtype='float64')

M, n = sigma.shape
R = np.array([5, 4])

num_policies = np.prod(R-1)
policies = enumerate_policies(M, R)
pi_pools, pi_policies = extract_pools(policies, sigma)

for pool_id, pool in pi_pools.items():
    print(pool_id, ":", [policies[i] for i in pool])

0 : [(1, 1), (2, 1)]
1 : [(1, 2), (1, 3), (2, 2), (2, 3)]
2 : [(3, 1), (4, 1)]
3 : [(3, 2), (3, 3), (4, 2), (4, 3)]


In [193]:
counter.num_pools(sigma)

4.0

In [194]:
counter.num_admissible_poolings(4, M, R)

6

In [195]:
np.random.seed(3)

num_pools = len(pi_pools)
mu = np.random.uniform(0, 4, size=num_pools)
var = [1] * num_pools

n_per_pol = 10

num_data = num_policies * n_per_pol
X = np.ndarray(shape=(num_data, M))
D = np.ndarray(shape=(num_data, 1), dtype='int_')
y = np.ndarray(shape=(num_data, 1))

for idx, policy in enumerate(policies):
    pool_i = pi_policies[idx]
    mu_i = mu[pool_i]
    var_i = var[pool_i]
    y_i = np.random.normal(mu_i, var_i, size=(n_per_pol, 1))

    start_idx = idx * n_per_pol
    end_idx = (idx + 1) * n_per_pol

    X[start_idx:end_idx, ] = policy
    D[start_idx:end_idx, ] = idx
    y[start_idx:end_idx, ] = y_i

In [196]:
policy_means = loss.compute_policy_means(D, y, num_policies)
policy_means

array([[22.3306393 , 10.        ],
       [26.09289987, 10.        ],
       [26.9056259 , 10.        ],
       [14.77744099, 10.        ],
       [27.87404021, 10.        ],
       [29.12185777, 10.        ],
       [14.06728243, 10.        ],
       [21.12531734, 10.        ],
       [15.25551251, 10.        ],
       [12.69909817, 10.        ],
       [21.11531527, 10.        ],
       [19.32885624, 10.        ]])

In [197]:
Q = loss.compute_Q(D, y, sigma, policies, policy_means, reg=1)
Q

4.9889861406550375

In [198]:
i = 1
j = 1
B = loss.compute_B(D, y, sigma, i, j, policies, policy_means, reg=1)
print(B)

6.975889428946971


In [199]:
P_set = RAggregate(2, R, 4, D, y, 1.4, reg=0.1)
print(P_set.size)
P_set.seen(sigma)

5


True